In [2]:
import bs4
import jdatetime
import json
import re
import os
import requests
import urllib3
import tqdm
import wget
import pandas as pd

In [3]:
base_url='https://trvotes.parliran.ir'

In [4]:
with open("index.html", 'r',encoding='utf8') as f:
    webpage = f.read()

In [4]:
soup = bs4.BeautifulSoup(webpage)

In [5]:
votes=soup.find_all('tr')[1:]

In [6]:
votes_parsed=[]
for vote in votes:
    title,time,url=vote.findAll('td')
    title=' '.join( title.text.split())
    time=time.text.replace('\n','').replace(' ','')
    # time=jdatetime.datetime.strptime(time,'%Y/%m/%d')
    url=base_url+url.find('a')['href']
    votes_parsed.append({'title':title,'time':time,'url':url})

In [7]:
with open('parsed.json','w',encoding='utf8') as f:
    t=json.dumps(votes_parsed,ensure_ascii=False)
    f.write(t)

In [38]:
members=dict()
for page in os.listdir('pages/'):
    with open(f"pages/{page}", 'r',encoding='utf8') as f:
        webpage = f.read()
        webpage.replace("\\u200c",'')
        print(page)
        soup = bs4.BeautifulSoup(webpage)
        break
        

10.html


In [39]:
stat_dict={
            '----':'absence',
            'عدم مشارکت':"non-participant",
            'مخالف':'against',
            'موافق':'favor',
            'ممتنع':'abstaining'
        }
def get_day_data(soup,date_:str):
    vote_title=soup.select_one('#page-wrapper > div.row > div.col-lg-12 > div > div.panel-footer').text
    vote_title=' '.join( vote_title.split())
    stats=soup.select('.inner h3')
    favor,against,abstaining ,members_count=map(lambda x: x.text,stats)
    data=soup.find_all('tr')[1:]
    votes=[]
    for row in data:
        row=row.find_all('th')

        img_url=row[0].find('img')['src']
        mem_id=img_url.split('/')[-1].split('.')[0]
        if os.path.exists(f'images/{img_url.split("/")[-1]}')== False:
            wget.download(base_url+img_url,f'images/{img_url.split("/")[-1]}')
        family_city=' '.join(row[2].text.split())
        paran=family_city.index('(')

        

        d={
        'img_url':img_url,
        'id':mem_id,
        'name':' '.join(row[1].text.split()),
        'date':date_,
        'family_city':family_city,
        'family':family_city[:paran],
        'city':family_city[paran:].replace('(','').replace(')',''),
        'stat':stat_dict[ ' '.join(row[4].text.split())]
        }
        if d['id'] not in members:
            members[d['id']]=[
                d['name'],
                d['family'],
                d['city']]
        votes.append(d)
    return [favor,against,abstaining,date_ ,members_count,votes]
get_day_data(soup,'//')

['123',
 '46',
 '8',
 '//',
 '215',
 [{'img_url': '/Content/deputyimg/11004.jpg',
   'id': '11004',
   'name': 'الهام',
   'date': '//',
   'family_city': 'آزاد (نايين)',
   'family': 'آزاد ',
   'city': 'نايين',
   'stat': 'absence'},
  {'img_url': '/Content/deputyimg/11005.jpg',
   'id': '11005',
   'name': 'احد',
   'date': '//',
   'family_city': 'آزادي\u200cخواه (ملاير)',
   'family': 'آزادي\u200cخواه ',
   'city': 'ملاير',
   'stat': 'against'},
  {'img_url': '/Content/deputyimg/11007.jpg',
   'id': '11007',
   'name': 'محمدحسن',
   'date': '//',
   'family_city': 'آصفري (اراك)',
   'family': 'آصفري ',
   'city': 'اراك',
   'stat': 'against'},
  {'img_url': '/Content/deputyimg/11008.jpg',
   'id': '11008',
   'name': 'حبيب',
   'date': '//',
   'family_city': 'آقاجري (بندماهشهر)',
   'family': 'آقاجري ',
   'city': 'بندماهشهر',
   'stat': 'absence'},
  {'img_url': '/Content/deputyimg/11017.jpg',
   'id': '11017',
   'name': 'محمدرضا',
   'date': '//',
   'family_city': 'احمدي (رش

In [22]:
soup.find_all('tr')[1:].__len__()

79